In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
#import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import gensim
from gensim.utils import simple_preprocess
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from nltk.cluster import KMeansClusterer, euclidean_distance
import nltk
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import seaborn as sns
from wordcloud import WordCloud
from nltk.tokenize import wordpunct_tokenize
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
#import flair
from scipy.spatial import distance
#from simpletransformers.language_representation import RepresentationModel
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
#import umap.umap_ as umap
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score
from wordcloud import WordCloud
#import texthero as hero
from sklearn import metrics
%matplotlib inline

In [ ]:
!pip install -U sentence-transformers
#!pip install "gensim==3.8.1"
#!pip install texthero
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.9 MB/s 
     |████████████████████████████████| 4.2 MB 51.5 MB/s 
     |████████████████████████████████| 1.2 MB 47.2 MB/s 
     |████████████████████████████████| 86 kB 3.6 MB/s 
     |████████████████████████████████| 596 kB 60.6 MB/s 
     |████████████████████████████████| 6.6 MB 54.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=9ff963f03b5178ec5b5a1daf3435874d855b7cacacd0933d762f89fdee846c13
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_train=pd.read_csv('/content/drive/MyDrive/BBC News Train.csv')
#data_test=pd.read_csv('/content/drive/MyDrive/BBC News Test.csv')
#data.head()

In [ ]:
data_train.drop(['ArticleId'],inplace=True,axis=1)
#data_test.drop(['ArticleId'],inplace=True,axis=1)

In [ ]:
data_train.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [ ]:
data_train.describe()

,Text,Category
count,1490,1490
unique,1440,5
top,microsoft seeking spyware trojan microsoft is ...,sport
freq,2,346


In [ ]:
vec = TfidfVectorizer()
features= vec.fit_transform(data_train['Text'])
print("Input features shape:", features.shape)
print(f"\nTake a look at the features extracted from the first news article:\n{features[0].toarray()}")

Input features shape: (1490, 24746)

Take a look at the features extracted from the first news article:
[[0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(5, n_init=10, random_state=42)
kmeans.fit(features)
data_train['cluster'] = kmeans.labels_

In [ ]:
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    print(f"{cat}\n{data_train[mark]['cluster'].value_counts()}\n")

business
3    168
1    163
2      4
4      1
Name: cluster, dtype: int64

tech
2    226
1     32
0      3
Name: cluster, dtype: int64

politics
4    164
1    107
2      2
3      1
Name: cluster, dtype: int64

sport
1    346
Name: cluster, dtype: int64

entertainment
0    129
1    129
2     10
3      5
Name: cluster, dtype: int64



In [ ]:
cluster_to_category = {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category

business:
 Top cluster number: 3, Number of samples: 168
tech:
 Top cluster number: 2, Number of samples: 226
politics:
 Top cluster number: 4, Number of samples: 164
sport:
 Top cluster number: 1, Number of samples: 346
entertainment:
 Top cluster number: 0, Number of samples: 129

Map cluster number to category:


{0: 'entertainment', 1: 'sport', 2: 'tech', 3: 'business', 4: 'politics'}

In [ ]:
data_train['clustered_category'] = data_train['cluster'].map(cluster_to_category)
#data_train.head(20)

In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category']))

Overall accuracy of clustered categories: 0.6932885906040268


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features,data_train['cluster']))

Silhouette Coefficient: 0.002


#Tf-idf-Preprocessed Data + K-means


In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import gensim
from gensim.utils import simple_preprocess
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from nltk.cluster import KMeansClusterer, euclidean_distance
import nltk
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import seaborn as sns
from wordcloud import WordCloud
from nltk.tokenize import wordpunct_tokenize
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('omw-1.4')
data_train=pd.read_csv('/content/drive/MyDrive/BBC News Train.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [ ]:
STOPWORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [ ]:
data_train.drop(['ArticleId'],axis=1,inplace=True)

In [ ]:
data_train.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [ ]:
def expand_contractions(text):
    expanded_words = []    
    for word in text.split():
        expanded_words.append(contractions.fix(word))   
    expanded_text = ' '.join(expanded_words)
    return expanded_text
 
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
def remove_digits(text):
    return " ".join([word for word in str(text).split() if not(word.isdigit())])
def remove_shorttokens(text):
    return " ".join([word for word in str(text).split() if len(word)>1])
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
#convert it to string
data_train["Text"]=data_train["Text"].astype(str)
#Convert to lower and strip
data_train["Text"]=data_train["Text"].str.lower().str.strip()
#apply contractions
data_train["Text"]=data_train["Text"].apply(lambda x:expand_contractions(x))
#remove punctuations
data_train["Text"]=data_train["Text"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
#remove stop words
data_train["Text"]=data_train["Text"].apply(lambda x: remove_stopwords(x))
#Lemmatize the sentence
data_train["Text"]=data_train["Text"].apply(lambda text: lemmatize_words(text))
data_train["Text"]=data_train["Text"].apply(lambda text:remove_digits(text))
data_train["Text"]=data_train["Text"].apply(lambda text:remove_shorttokens(text))

In [ ]:
data_train.head()

,Text,Category
0,worldcom exboss launch defence lawyer defendin...,business
1,german business confidence slide german busine...,business
2,bbc poll indicates economic gloom citizen majo...,business
3,lifestyle governs mobile choice faster better ...,tech
4,enron boss 168m payout eighteen former enron d...,business


In [ ]:
vec = TfidfVectorizer()
features= vec.fit_transform(data_train['Text'])
print("Input features shape:", features.shape)
print(f"\nTake a look at the features extracted from the first news article:\n{features[0].toarray()}")

Input features shape: (1490, 23815)

Take a look at the features extracted from the first news article:
[[0. 0. 0. ... 0. 0. 0.]]


scipy.sparse.csr.csr_matrix

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(5, n_init=10, random_state=42)
kmeans.fit(features)
data_train['cluster'] = kmeans.labels_

In [ ]:
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    print(f"{cat}\n{data_train[mark]['cluster'].value_counts()}\n")

business
0    322
2      5
1      5
4      3
3      1
Name: cluster, dtype: int64

tech
1    223
0     17
2     13
3      8
Name: cluster, dtype: int64

politics
4    200
0     67
2      4
1      3
Name: cluster, dtype: int64

sport
2    345
3      1
Name: cluster, dtype: int64

entertainment
3    241
0     22
2      6
1      4
Name: cluster, dtype: int64



In [ ]:
cluster_to_category = {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category

business:
 Top cluster number: 0, Number of samples: 322
tech:
 Top cluster number: 1, Number of samples: 223
politics:
 Top cluster number: 4, Number of samples: 200
sport:
 Top cluster number: 2, Number of samples: 345
entertainment:
 Top cluster number: 3, Number of samples: 241

Map cluster number to category:


{0: 'business', 1: 'tech', 2: 'sport', 3: 'entertainment', 4: 'politics'}

In [ ]:
data_train['clustered_category'] = data_train['cluster'].map(cluster_to_category)
data_train.head(20)

,Text,Category,cluster,clustered_category
0,worldcom exboss launch defence lawyer defendin...,business,0,business
1,german business confidence slide german busine...,business,0,business
2,bbc poll indicates economic gloom citizen majo...,business,0,business
3,lifestyle governs mobile choice faster better ...,tech,1,tech
4,enron boss 168m payout eighteen former enron d...,business,0,business
5,howard truanted play snooker conservative lead...,politics,2,sport
6,wale silent grand slam talk rhys williams say ...,sport,2,sport
7,french honour director parker british film dir...,entertainment,3,entertainment
8,car giant hit mercedes slump slump profitabili...,business,0,business
9,fockers fuel festive film chart comedy meet fo...,entertainment,3,entertainment


In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category']))

Overall accuracy of clustered categories: 0.8932885906040269


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features,data_train['cluster']))

Silhouette Coefficient: 0.014


#Experiment with PCA


In [ ]:
list_features=features.toarray()
print(type(list_features))
#list_features.shape()

<class 'numpy.ndarray'>


In [ ]:
data_embbedding=list_features
x=[]
y=[]
for i in range(1490,0,-50):
  pca_2 = PCA(n_components=i)
  pca_2_result = pca_2.fit_transform(data_embbedding)
  x.append(i)
  y.append(round(np.sum(pca_2.explained_variance_ratio_),2))
  print('Cumulative variance explained by {} principal components: {:.2%}'.format(i,np.sum(pca_2.explained_variance_ratio_)))

Cumulative variance explained by 1490 principal components: 100.00%
Cumulative variance explained by 1440 principal components: 100.00%
Cumulative variance explained by 1390 principal components: 99.82%
Cumulative variance explained by 1340 principal components: 99.07%
Cumulative variance explained by 1290 principal components: 98.10%
Cumulative variance explained by 1240 principal components: 97.00%
Cumulative variance explained by 1190 principal components: 95.53%
Cumulative variance explained by 1140 principal components: 94.11%
Cumulative variance explained by 1090 principal components: 92.57%
Cumulative variance explained by 1040 principal components: 90.91%
Cumulative variance explained by 990 principal components: 89.12%
Cumulative variance explained by 940 principal components: 87.20%
Cumulative variance explained by 890 principal components: 85.14%
Cumulative variance explained by 840 principal components: 82.96%
Cumulative variance explained by 790 principal components: 80.65

In [ ]:
#95%,90%,85% covaience-->1160,1010,885
vec_95 = TfidfVectorizer(
        max_df=0.5,
        min_df=2,
        stop_words="english",
        max_features=1160,
        use_idf=True
    )
vec_90 = TfidfVectorizer(
        max_df=0.5,
        min_df=2,
        stop_words="english",
        max_features=1010,
        use_idf=True
    )
vec_85 = TfidfVectorizer(
        max_df=0.5,
        min_df=2,
        stop_words="english",
        max_features=885,
        use_idf=True
    )
features95= vec_95.fit_transform(data_train['Text'])
features90=vec_90.fit_transform(data_train['Text'])
features85=vec_85.fit_transform(data_train['Text'])


In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(5, n_init=10, random_state=42)
kmeans.fit(features95)
data_train['cluster95'] = kmeans.labels_

kmeans = KMeans(5, n_init=10, random_state=42)
kmeans.fit(features90)
data_train['cluster90'] = kmeans.labels_

kmeans = KMeans(5, n_init=10, random_state=42)
kmeans.fit(features85)
data_train['cluster85'] = kmeans.labels_

In [ ]:
cluster_to_category95= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster95'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category95[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category95

business:
 Top cluster number: 1, Number of samples: 311
tech:
 Top cluster number: 0, Number of samples: 249
politics:
 Top cluster number: 2, Number of samples: 243
sport:
 Top cluster number: 4, Number of samples: 344
entertainment:
 Top cluster number: 3, Number of samples: 136

Map cluster number to category:


{0: 'tech', 1: 'business', 2: 'politics', 3: 'entertainment', 4: 'sport'}

In [ ]:
data_train['clustered_category95'] = data_train['cluster95'].map(cluster_to_category95)

In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category95']))

Overall accuracy of clustered categories: 0.8610738255033556


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features95,data_train['cluster95']))

Silhouette Coefficient: 0.029


In [ ]:
cluster_to_category90= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster90'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category90[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category90

business:
 Top cluster number: 1, Number of samples: 312
tech:
 Top cluster number: 0, Number of samples: 247
politics:
 Top cluster number: 2, Number of samples: 241
sport:
 Top cluster number: 4, Number of samples: 345
entertainment:
 Top cluster number: 3, Number of samples: 136

Map cluster number to category:


{0: 'tech', 1: 'business', 2: 'politics', 3: 'entertainment', 4: 'sport'}

In [ ]:
data_train['clustered_category90'] = data_train['cluster90'].map(cluster_to_category90)

In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category90']))

Overall accuracy of clustered categories: 0.859731543624161


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features90,data_train['cluster90']))

Silhouette Coefficient: 0.030


In [ ]:
cluster_to_category85= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster85'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category85[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category85

business:
 Top cluster number: 4, Number of samples: 312
tech:
 Top cluster number: 0, Number of samples: 247
politics:
 Top cluster number: 1, Number of samples: 241
sport:
 Top cluster number: 3, Number of samples: 344
entertainment:
 Top cluster number: 2, Number of samples: 131

Map cluster number to category:


{0: 'tech', 1: 'politics', 2: 'entertainment', 3: 'sport', 4: 'business'}

In [ ]:
data_train['clustered_category85'] = data_train['cluster85'].map(cluster_to_category85)

In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category85']))

Overall accuracy of clustered categories: 0.8557046979865772


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features85,data_train['cluster85']))

Silhouette Coefficient: 0.032
